<a href="https://colab.research.google.com/github/chaithrashagu/Internship/blob/master/Sentimental%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentimental Analysis: Movie Review Analysis**

**What is sentimental analysis?**

>      Sentimental Analysis is the automated process of understanding the opinion about a given subject from a written or spoken language.

> **Usage of sentimental analysis**

1.   Product Reviews
2.   Movie Reviews
3.   Food Reviews
4.   Agricultural Reviews

**INTRODUCTION:**

> Our  analysis helps concerned organization to find opinions of people about movies from their reviews, if it is Positive, Negative or Netrual.

> Our goal is to calculate the polarity of sentences that we extract  from the text of reviews.

> We use the Internet Movies Database(IMDB) movie review datasets.


















In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pop=pd.read_csv('sampleSubmission.csv')
pop.head()


,PhraseId,Sentiment
0,156061,2
1,156062,2
2,156063,2
3,156064,2
4,156065,2


In [0]:
train= open('train.tsv', 'r') # Training data
train.readline() # Read and remove header row
word_sentiment = {}

In [0]:
def simplify(doc):
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A).lower().strip() # Remove special characters, whitespaces and make lower case
    tokens = nltk.WordPunctTokenizer().tokenize(doc) # Tokenize
    filtered_tokens = [token for token in tokens if token not in nltk.corpus.stopwords.words('english')] # Remove stopwords
    doc = ' '.join(filtered_tokens) # Re-create document from filtered tokens
    return doc

In [0]:
def get_words(sentence):
    stemmer = PorterStemmer()
    words = [stemmer.stem(x) for x in simplify(sentence).split()]
    return words

In [25]:
for data in train:
    sentiment, line = data.split(',')
    words = get_words(line)
    for word in words:
        try: # Increment weight of the word by 1 if positive, decrement weight by 1 if negative
            word_sentiment[word][0] = word_sentiment[word][0] + 1 if int(sentiment) == 1 else word_sentiment[word][0] - 1
        except: # If word doesn't exist, create new entry in dictionary
            word_sentiment[word] = [1, 0] if int(sentiment) == 1 else [-1, 0]
        finally: # Increment number of occurences of the word; used later to compute the weighted sum instead of just sum
            word_sentiment[word][1] += 1

NameError: ignored